In [1]:
import os
from PIL import Image
from PIL.ExifTags import TAGS
import pandas as pd

In [2]:
def extraire_metadata(image_path):
    """Retourne un dictionnaire contenant les métadonnées EXIF de l'image."""
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
    except Exception:
        return {}

    metadata = {}
    if exif_data:
        for tag_id, value in exif_data.items():
            tag = TAGS.get(tag_id, tag_id)
            metadata[tag] = value

    return metadata

In [3]:
# --- Chemin du dossier contenant les images ---
dossier_images = "/media/lecaotunglam/ESD-USB/PHOTOS BALADE/"

# --- Extensions acceptées ---
extensions = (".jpg", ".jpeg", ".png", ".tiff", ".bmp")

liste_metadonnees = []

for fichier in os.listdir(dossier_images):
    if fichier.lower().endswith(extensions):
        chemin = os.path.join(dossier_images, fichier)
        meta = extraire_metadata(chemin)

        # Ajout du nom du fichier
        meta["Fichier"] = fichier

        liste_metadonnees.append(meta)

# --- Conversion en tableau (DataFrame) ---
df = pd.DataFrame(liste_metadonnees)

# --- Export CSV ---
df.to_csv("metadata_images.csv", index=False, encoding='utf-8')

In [5]:
df.to_csv("/home/lecaotunglam/PycharmProjects/PythonMetaData/metadata_images.csv", index=False, encoding='utf-8')

In [ ]:
import piexif

image_path = "input.jpg"

# Load existing EXIF
exif_dict = piexif.load(image_path)

# Modify some tags
exif_dict["0th"][piexif.ImageIFD.Artist] = "John Doe"
exif_dict["0th"][piexif.ImageIFD.Make] = "MyCameraBrand"
exif_dict["Exif"][piexif.ExifIFD.DateTimeOriginal] = "2025:01:01 12:00:00"

# Save back to image
exif_bytes = piexif.dump(exif_dict)
piexif.insert(exif_bytes, image_path)

print("✔ Metadata updated!")


In [ ]:
import piexif

image = "input.jpg"
exif = piexif.load(image)

# GPS format: (numerator, denominator)
def to_deg(value):
    deg = int(value)
    min = int((value - deg) * 60)
    sec = int(((value - deg) * 60 - min) * 60)
    return [(deg, 1), (min, 1), (sec, 1)]

lat = 48.8566   # Paris
lon = 2.3522

exif["GPS"][piexif.GPSIFD.GPSLatitudeRef] = "N"
exif["GPS"][piexif.GPSIFD.GPSLatitude] = to_deg(lat)
exif["GPS"][piexif.GPSIFD.GPSLongitudeRef] = "E"
exif["GPS"][piexif.GPSIFD.GPSLongitude] = to_deg(lon)

exif_bytes = piexif.dump(exif)
piexif.insert(exif_bytes, image)

print("✔ GPS metadata added!")


In [ ]:
import pyexiv2

image = pyexiv2.Image("input.jpg")
image.readMetadata()

image["Exif.Image.Artist"] = "Jane Doe"
image["Iptc.Application2.Caption"] = ["New caption here"]
image["Xmp.dc.title"] = "My Title"

image.writeMetadata()

print("✔ IPTC/XMP updated!")
